# build name signs

In [ ]:
# PAPERMILL PARAMETERS
FILE_SECRET = ""
FILE_OUTPUT = ""
SHEET = ""
TAB = ""

In [ ]:
# Injected parameters
FILE_SECRET = ""
FILE_OUTPUT = ""
SHEET = ""
TAB = ""

In [ ]:
import os

import gspread
import gspread_dataframe
import numpy as np
import oauth2client.service_account
import pandas as pd
from fpdf import FPDF

In [ ]:
class PDF(FPDF):
    # 210 x 297 mm
    def lines(self):
        self.set_line_width(0.25)
        self.line(0, 105, 5, 105)
        self.line(292, 105, 297, 105)

    def last_name(self, x):
        self.set_xy(0, 170)
        self.set_font("DejaVu", "B", 50)
        self.set_text_color(100, 100, 100)
        self.cell(w=297, align="C", txt=x, border=0)

    def first_name(self, x):
        self.set_xy(0, 125)
        self.set_font("DejaVu", "B", 120)
        self.set_text_color(0, 0, 0)
        self.cell(w=297, align="C", txt=x, border=0)


def read_sheet(id, tab, gc):
    # read data
    data_sheet = gc.open_by_key(id).worksheet(tab)
    data = data_sheet.get_all_values()

    # extract header
    headers = data[0]
    data = data[1:]

    return pd.DataFrame(
        data,
        columns=["_".join(x.lower().split(" ")) for x in headers],
    )

## read data

In [ ]:
credentials = (
    oauth2client.service_account.ServiceAccountCredentials.from_json_keyfile_name(
        os.path.expandvars(FILE_SECRET),
        "https://spreadsheets.google.com/feeds",
    )
)
gc = gspread.authorize(credentials)

In [ ]:
data = read_sheet(
    SHEET,
    TAB,
    gc,
)

In [ ]:
data.head(3)

## create name signs

In [ ]:
pdf = PDF(orientation="L", unit="mm", format="A4")
pdf.add_font(family="DejaVu", style="B", fname="DejaVuSans-Bold.ttf")
pdf.set_font("DejaVu", "B")

for i in range(data.shape[0]):
    pdf.add_page()
    first_name = data.iloc[i]["first_name"]
    last_name = data.iloc[i]["last_name"]
    pdf.lines()
    pdf.first_name(first_name)
    pdf.last_name(last_name)

    with pdf.rotation(180, 297 / 2, 210 / 2):
        pdf.first_name(first_name)
        pdf.last_name(last_name)

pdf.output(FILE_OUTPUT)